# 
## Problem

A friend recently told me she was leaving the salon where she had worked for many years in order to open her own business. Her target clients are young, urban, professional women. For that reason, she explained, she wanted to find a location where customers spend money: not a strictly residential area of her city, nor an area with strictly office buildings or industries.

I decided to use this as inspiration for the capstone project. I can leverage the Foursquare location data to find a data-driven recommendation for where to search for real estate for a new salon. This recommendation would help the stakeholder (i.e., the new business owner) find a target location where there is neither a superfluity of salons nor a dearth of disposable income being spent. 




## Data

To answer this question, I will use Foursquare location data for Toronto. I will get 100 venues from Foursquare for each postal code in Toronto, as we did in the previous assignment. Then, I will use the venue categories of this location data to determine what postal code has low competition (no other salons in the postal code) but high disposable income being spent. 

To operationalize this second component, I will use restaurants as a stand-in for 'disposable income being spent'. As the cliché goes, "location is everything" in the restaurant industry: dense pockets of restaurants indicate that there is enough clientele to keep these restaurants in business. 

The data cleaning will include joining multiple related "Venue Categories" together. For example, 'BBQ Joint' and 'Sandwich Place' can both be grouped into a super-category of 'Restaurants'; while some salons are categorized as a 'Salon / Barbershop', others are categorized as a 'Health & Beauty Service'. 

The machine learning method of DBSCAN (density-based spatial clustering of applications with noise) will help me visualize the areas of high restaurant frequency. This method will group restaurants that have many neighbors and will de-prioritize isolated restaurants as outliers. By cross-referencing these clusters with the location of salons, I will be able to determine which clusters currently lack a salon, and therefore, which neighborhood is a promising area for a new salon owner to search for real estate.
